**<h1>Welcome to Telegram Broadcaster Bot!</h1>**

It is a mass messaging tool for organisations.

In [ ]:
# @title <h1>Install Packages</h1>
!pip install python-telegram-bot==20.6
!pip install psycopg2-binary==2.9.9
!pip install pandas==1.5.3
!pip install sqlalchemy==1.4.50
!pip install awscli==1.31.12

import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import OperationalError
import telegram
from telegram import Bot
import asyncio
import logging
from time import time
from sys import exit
from asyncio import sleep, Lock
from google.colab import output, userdata
from io import StringIO
from telegram.ext import ApplicationBuilder, BaseRateLimiter
from telegram.request import HTTPXRequest
import threading
import subprocess

output.clear()

In [ ]:
#time.sleep(2)

# @title <h1>Input file name of recipient list</h1> { form-width: "50%" }
recipient_list_filename = "recipient_list.xlsx" # @param {type:"string"}
db_params = {
     'dbname':userdata.get('user'),
     'user':userdata.get('user'),
     'password':userdata.get('password'),
     'host':"host",
     'port':"port"
    }

db_uri = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

engine = create_engine(db_uri)
query = "SELECT telegram_id, phone_number FROM subscriber_list"
subscriber_list = pd.read_sql(query, engine)

subscriber_list['phone_number'] = subscriber_list['phone_number'].apply(lambda x: str(x).replace("-", "").strip())
subscriber_list.rename(columns={'phone_number': 'Mobile No'}, inplace=True)

# #read recepient spreadsheet
df_recipient_list = pd.read_excel(recipient_list_filename)
df_recipient_list['Mobile No'] = df_recipient_list['Mobile No'].apply(lambda x: str(x).replace("-", "").strip())

# # Join the dataframes on the phone_numbers column
merged_df = pd.merge(df_recipient_list, subscriber_list, on='Mobile No', how='inner')

<h1>Upload Recipient List</h1>

Find the icon on the left of your screen to upload your Recipient List.

<div align="right">
    <img src="https://blog.roboflow.com/content/images/2020/10/upload.png" />
</div>

<h1>Create Message Template</h1>

To personalise your message, include keywords that are surrounded by {curly braces}. <u>**The keywords in your message template should match the headers in your recipients excel file.**</u>

```
Dear {Name}, your mobile number is {Mobile No} and your Rank is {Rank}.
```

To format your message, refer to the following:

```
<b>bold</b>

<i>italic</i>

<u>underline</u>

<s>strikethrough</s>

<span class="tg-spoiler">spoiler</span>

<a href="http://www.example.com/">inline URL</a>
```



<H1>DOUBLE CLICK ME TO FORMAT YOUR TEXT</H1>
<h2>PREVIEW TO YOUR RIGHT</h2>
<H3>When you're done, just paste the entire block into the message_template below</H3>
===================================================================================================================================

📣<b><u> PREVIEW</u></b>

<b>bold</b>

<i>italic</i>

<u>underline</u>

<s>strikethrough</s>

<span class="tg-spoiler">spoiler</span>

<a href="http://www.example.com/">inline URL</a>




In [ ]:
# @title <h1>Create Message_Template</h1>

message_template = """📝 Test Message

Dear {Name},

this is bold

this is italic

underline

strikethrough

this is new! - Click on the moving dots to reveal - spoiler effect


You should not be able to copy or forward this message

"""

In [ ]:
# @title <h1>Enter your Bot Token</h1>
your_bot_token = input("Please enter your bot token:")
output.clear()

In [ ]:
# @title <h1>Send a preview message</h1>
test_number = "xxxxxxxxxx" # @param {type:"string"}

# SEND TEST MESSAGE
# Initialize logging with a custom handler
logger = logging.getLogger()

async def main():

    bot = Bot(token=your_bot_token)

    if test_number in subscriber_list['Mobile No'].values:
      for index, row in subscriber_list.iterrows():
        if row['Mobile No'] == test_number:

          row_dict = row.to_dict()
          text = message_template.format(**row_dict)
          try:
            await bot.send_message(chat_id=row['telegram_id'], text=text, parse_mode='html')
            print('PREVIEW MESSAGE SUCCESSFUL')
          except telegram.error.BadRequest as e:
            logger.error(f"BadRequest error for {row['Mobile No']}: {e}")
          except telegram.error.TelegramError as e:
            logger.error(f"Telegram error for {row['Mobile No']}: {e}")
          except Exception as e:
            logger.error(f"An unexpected error occurred for {row['Mobile No']}: {e}")
    else:
      print('ERROR, test_number not found, please check again!')


# Run the main function
await main()


In [ ]:
# @title <h1>Send Campaign</h1>
from time import time
disable_forwarding = False # @param {type:"boolean"}

confirmation = input("You are about to send the campaign, enter Y to continue: ")
if confirmation == 'Y':
  print ('ok. Processing...')
  pass

else:
  print("Hit the play button again when you're ready")
  exit()

class ListHandler(logging.Handler):
    def __init__(self):
        super().__init__()
        self.records = []

    def emit(self, record):
        self.records.append(self.format(record))

# Initialize logging with a custom handler
logger = logging.getLogger()

list_handler = ListHandler()
logger.addHandler(list_handler)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
list_handler.setFormatter(formatter)
successful_messages = 0

# Custom rate limiter
class MyRateLimiter(BaseRateLimiter):
    def __init__(self):
        self.lock = asyncio.Lock()
        self.bucket_capacity = 20  # Max 20 tokens (messages)
        self.tokens = 20  # Starts full
        self.last_refill_time = time()
        self.refill_rate = 1 / 20  # 20 messages per second

    async def initialize(self) -> None:
        pass

    async def shutdown(self) -> None:
        pass

    async def process_request(
        self,
        callback,
        args,
        kwargs,
        endpoint,
        data,
        rate_limit_args
    ):
      async with self.lock:
        current_time = time()
        time_elapsed = current_time - self.last_refill_time
        self.last_refill_time = current_time
        self.tokens = min(self.bucket_capacity, self.tokens + time_elapsed / self.refill_rate)

        logger.info(f"Tokens available: {self.tokens}")

        if self.tokens < 1:
            sleep_time = (1 - self.tokens) * self.refill_rate
            logger.info(f"Sleeping for {sleep_time} seconds")
            await sleep(sleep_time)

        self.tokens -= 1
      return await callback(*args, **kwargs)

# Async function to send messages
async def send_message(bot, row):
    global successful_messages
    row_dict = row.to_dict()
    text = message_template.format(**row_dict)
    try:
        await bot.send_message(chat_id=row['telegram_id'], text=text, parse_mode='html', protect_content=disable_forwarding)
        successful_messages += 1
    except telegram.error.BadRequest as e:
        logger.error(f"BadRequest error for {row['Mobile No']}: {e}")
    except telegram.error.TelegramError as e:
        logger.error(f"Telegram error for {row['Mobile No']}: {e}")
    except Exception as e:
        logger.error(f"An unexpected error occurred for {row['Mobile No']}: {e}")

custom_request = HTTPXRequest(
    connection_pool_size=25,
    read_timeout=10.0,
    write_timeout=10.0,
    connect_timeout=10.0,
    pool_timeout=10.0,
    http_version='1.1'
)

# Initialize application builder
application = ApplicationBuilder().token(your_bot_token).rate_limiter(MyRateLimiter()).request(custom_request).build()
bot = application.bot

async def main():
    batch_size = 20  # Set the size of each batch
    for i in range(0, len(merged_df), batch_size):
        # Create a batch of tasks
        batch = merged_df[i:i + batch_size]
        tasks = [asyncio.ensure_future(send_message(bot, row)) for index, row in batch.iterrows()]
        # Wait for the entire batch of tasks to complete before starting the next
        await asyncio.gather(*tasks)
        logger.info(f"Batch {i // batch_size + 1} completed")

    print(f"Total successful messages sent: {successful_messages} out of {len(df_recipient_list)} ({successful_messages/len(df_recipient_list)*100}%)")

# Run the main function
await main()

# At the end of your script
log_records = [record.split(' - ') for record in list_handler.records]
error_logs_df = pd.DataFrame(log_records, columns=['Timestamp', 'Level', 'Message'])

# Filter for error logs only
error_logs_df = error_logs_df[error_logs_df['Level'] == 'ERROR']

In [ ]:
# @title <h1>Generate Report</h1>
merged_outer = pd.merge(df_recipient_list, subscriber_list, on='Mobile No', how='outer', indicator=True)
non_subscribers = merged_outer[merged_outer['_merge'] == 'left_only']

error_logs_df[['Type', 'Contact']] = error_logs_df['Message'].str.split("for", expand = True)
error_logs_df[['Contact', 'Error']] = error_logs_df['Contact'].str.split(":", 1, expand = True)
error_logs_df.drop(['Level'], axis=1, inplace=True)

non_subscribers.drop(['telegram_id','_merge'], axis=1, inplace=True)

with pd.ExcelWriter('campaign_report.xlsx') as writer:
  error_logs_df.to_excel(writer, sheet_name='Error Log')
  non_subscribers.to_excel(writer, sheet_name='Non-Subscribers')

<h1>Find your report in the column on the left</h1>

The report will be titled 'campaign_report.xlsx'. Right click to download!

<div align="right">
    <img src="https://i.stack.imgur.com/2fJaL.png" />
</div>

In [ ]:
# @title <h1>View List of Subscribers</h1>
merged_df

In [ ]:
# @title <h1>View List of Non-Subscribers</h1>
merged_outer = pd.merge(df_recipient_list, subscriber_list, on='Mobile No', how='outer', indicator=True)
non_subscribers = merged_outer[merged_outer['_merge'] == 'left_only']
non_subscribers

In [ ]:
subscriber_list['Mobile No']